### References
- https://jovian.ai/aakashns/01-pytorch-basics
- https://jovian.ai/aakashns/02-linear-regression

### Setup

In [1]:
import torch
import numpy as np
from IPython.display import display
import pandas as pd

### gradとは
- 微分対象にしたい変数に、requires_grad=Trueをする。
- これらを使用した演算結果は、backwardできるように勝手になる。
- backwardは誤差逆伝搬のこと。
- 機械学習のパラメータは、再急降下法により更新され、最適値を目指す。
- loss関数を各パラメータで偏微分した値でパラメータを更新する。
  - lossをy、parametersをX、更新後のparametersをX'とすると、更新式は以下となる。
  - X'[t] = X[t] - a * dy / dX[t]
  - ここで、aはいわゆるlearning_rateである。

In [2]:
x = torch.tensor(3.)
w = torch.tensor(4., requires_grad=True) # これで微分対象となる
b = torch.tensor(5., requires_grad=True) # これで微分対象となる
x, w, b

y = w * x + b
y

tensor(17., grad_fn=<AddBackward0>)

- くらえ！自動微分！！

In [3]:
y.backward()
display(f"dy/dx = {x.grad}, dy/dw = {w.grad}, dy/db = {b.grad}")

'dy/dx = None, dy/dw = 3.0, dy/db = 1.0'

- ちなみに微分前に、微分を覗くとNoneになっている。

In [4]:
x = torch.tensor(3.)
w = torch.tensor(4., requires_grad=True) # これで微分対象となる
b = torch.tensor(5., requires_grad=True) # これで微分対象となる
print(f"dy/dx = {x.grad}, dy/dw = {w.grad}, dy/db = {b.grad}")

dy/dx = None, dy/dw = None, dy/db = None


### 線形回帰
- 何かyを推定する際に、入力情報の線形結合でモデル化する場合、これを線形モデルという。
  - モデルとしてはこんな感じ。なんかの係数と入力ベクトルの和で表現されるなら全部線形モデル。
    - y' = b + a0 * x0 + a1 * x1 ...
  - よく、線形は１次関数とか言われますけど、２次関数も線形モデルです。
  - なぜなら、以下のような２次関数も入力の線形結合だから。
    - y' = b + a0 * x + a1 * x * x
- せっかくなんで、pandas使ってみる。

In [5]:
df = pd.DataFrame([[73, 67, 43, 56, 70], 
                   [91, 88, 64, 81, 101], 
                   [87, 134, 58, 119, 133], 
                   [102, 43, 37, 22, 37], 
                   [69, 96, 70, 103, 119]], columns=["温度", "降水量", "湿度", "apples", "oranges"])
display(df[["温度", "降水量", "湿度"]])

,温度,降水量,湿度
0,73,67,43
1,91,88,64
2,87,134,58
3,102,43,37
4,69,96,70


- 入力と正解をtensorにする。
  - pandas -> numpy -> tensorに変換する。

In [6]:
inputs = torch.from_numpy(df[["温度", "降水量", "湿度"]].values.astype(np.float32))
display(inputs.shape)

targets = torch.from_numpy(df[["apples", "oranges"]].values.astype(np.float32))
display(targets.shape)

torch.Size([5, 3])

torch.Size([5, 2])

- パラメータ初期化

In [7]:
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
display(w,b)

tensor([[ 2.4211,  0.8010,  1.7180],
        [-1.3150,  0.9908,  2.3860]], requires_grad=True)

tensor([-9.9870e-01, -1.5625e-04], requires_grad=True)

- モデル定義
  - 初期値はランダムなので、この時点で推論しても意味不明である。
  - .t()は転置になるらしい。

In [8]:
def model(x):
    return x @ w.t() + b

preds = model(inputs)
display(preds, targets)

tensor([[303.2856,  72.9860],
        [399.7655, 120.2286],
        [416.6189, 156.7496],
        [343.9658,  -3.2445],
        [363.2171, 171.4011]], grad_fn=<AddBackward0>)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

- 誤差関数定義
  - 学習の目標とする、最小化したい指標を定義する。
  - .numel()は要素数になるらしい。

In [9]:
# MSE ... Mean squared error
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

loss = mse(preds, targets)
display(loss)

tensor(42802.1797, grad_fn=<DivBackward0>)

- 再度、微分じゃ！くらえ！

In [10]:
loss.backward()

In [11]:
display(w) # これは前と一緒
display(w.grad)

tensor([[ 2.4211,  0.8010,  1.7180],
        [-1.3150,  0.9908,  2.3860]], requires_grad=True)

tensor([[24749.5703, 24665.1602, 15684.8203],
        [  708.9462,  1674.9237,   983.1072]])

- 微分を元に再急降下法の式で更新する。
- この更新計算時は、微分は不要なので、no_gradというcontextmanagerが必要。
- contextmanagerは自分で作成もできる。以下を参考。
  - https://qiita.com/QUANON/items/c5868b6c65f8062f5876

In [12]:
# 1e-5がlearning_rateである。
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

- 更新が終わったら、次の更新に備えて、gradをゼロに戻しておいた方が良い。

In [13]:
w.grad.zero_()
b.grad.zero_()
display(w.grad, b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])

tensor([0., 0.])

- 更新後の値で、再度推論をし、誤差を計算しなおして、誤差が減ることを確認します。

In [14]:
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(29404.9785, grad_fn=<DivBackward0>)


### 複数回のepochで学習
- 複数回実行する形式で振り返ります。
- トータルとしてこういう流れが必要です。
- 少し見た目もおしゃれにしました。

In [15]:
import time
from tqdm.notebook import tqdm

# 入力
inputs = torch.from_numpy(df[["温度", "降水量", "湿度"]].values.astype(np.float32))

# 出力(正解)
targets = torch.from_numpy(df[["apples", "oranges"]].values.astype(np.float32))

# パラメータ初期化
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)

# モデル定義
def model(x):
    return x @ w.t() + b

# 誤差関数定義
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

# Train for 100 epochs
with tqdm(range(100)) as pbar:
    for i in pbar:
        time.sleep(0.05)
        preds = model(inputs)
        loss = mse(preds, targets)
        loss.backward()
        with torch.no_grad():
            w -= w.grad * 1e-5
            b -= b.grad * 1e-5
            w.grad.zero_()
            b.grad.zero_()
        
        pbar.set_description(f"[loss: {loss:.1f}]")

  0%|          | 0/100 [00:00<?, ?it/s]

- predsとtargetsの値を比較してみます。

In [16]:
display(preds, targets)

tensor([[ 59.6970,  75.3024],
        [ 87.3153,  87.3418],
        [103.0003, 155.2428],
        [ 37.4912,  64.6332],
        [100.9570,  79.7776]], grad_fn=<AddBackward0>)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [18]:
import jovian
jovian.commit(project='pytorch-tutorial-02-linear-model', filename='02_linear_model.ipynb')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[jovian] Creating a new project "nokomoro3/pytorch-tutorial-02-linear-model"
[jovian] Committed successfully! https://jovian.ai/nokomoro3/pytorch-tutorial-02-linear-model


'https://jovian.ai/nokomoro3/pytorch-tutorial-02-linear-model'